In [ ]:
from easydict import EasyDict as ED
import torch
# ^^^ pyforest auto-imports - don't write above this line
from data_reader import CINC2021Reader as DR
from cfg import TrainCfg, TrainCfg_ns, ModelCfg, ModelCfg_ns
from model import ECG_CRNN_CINC2021
from dataset import CINC2021

from tqdm.auto import tqdm
from copy import deepcopy

from official_phase_legacy.cfg import TrainCfg as TrainCfg_old
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

%load_ext autoreload
%autoreload 2

In [ ]:
from trainer import CINC2021Trainer

In [ ]:
ECG_CRNN_CINC2021.__DEBUG__ = False

In [ ]:
CINC2021.__DEBUG__ = False

In [ ]:
CINC2021Trainer.__DEBUG__ = True

In [ ]:
TrainCfg_ns

In [ ]:
ds_train = CINC2021(TrainCfg_ns, training=True, lazy=False)
ds_val = CINC2021(TrainCfg_ns, training=False, lazy=False)

In [ ]:
ds_train[1]

In [ ]:
TrainCfg_ns_nn = deepcopy(TrainCfg_ns)
TrainCfg_ns_nn.pop("normalize")

In [ ]:
ds_train_nn = CINC2021(TrainCfg_ns_nn, training=True, lazy=False)
ds_val_nn = CINC2021(TrainCfg_ns_nn, training=False, lazy=False)

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle, no normalization (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns_nn)
train_config.rnn_name = "lstm"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train_nn, ds_val_nn)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()

In [ ]:
del model

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle, no normalization, no mixup (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns_nn)
train_config.rnn_name = "lstm"

train_config.pop("mixup")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train_nn, ds_val_nn)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()

## 12 lead, resnet_nature_comm_se, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

## 12 lead, TResNet-F, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "tresnetF"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, TResNet-N, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "tresnetN"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, multi-scopic, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "multi_scopic"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, multi-scopic-leadwise, lstm, se, 2-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.cnn_name = "multi_scopic_leadwise"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, TResNet-P, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "tresnetP"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, multi-scopic-leadwise, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "multi_scopic_leadwise"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linear, lr=1e-3 (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.lr_scheduler = "none"
train_config.learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, resnet_nature_comm_se, no lstm, no se, 2-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linear, no lstm, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
# train_config.rnn_name = "lstm"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer

In [ ]:
import logging
logging.shutdown()

## 12 lead, multi_scopic, lstm, se, 2-linear, AsymmetricLoss, lr=1e-4 to 1e-3, one_cycle (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.cnn_name = "multi_scopic"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()

## 12 lead, resnet_nature_comm_bottleneck_se, no lstm, no se, 1-linear, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle(finished)

In [ ]:
from torch_ecg.torch_ecg.model_configs.cnn import resnet_nature_comm_bottle_neck_se

train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "none"
train_config.attn_name = "none"
train_config.cnn_name = "resnet_nature_comm_bottle_neck_se"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.cnn.resnet_nature_comm_bottle_neck_se = deepcopy(resnet_nature_comm_bottle_neck_se)
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

model_config.clf = ED()
model_config.clf.out_channels = [
  # not including the last linear layer, whose out channels equals n_classes
]
model_config.clf.bias = True
model_config.clf.dropouts = 0.0
model_config.clf.activation = "mish"  # for a single layer `SeqLin`, activation is ignored

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 12 lead, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle, no normalization, no mixup, with label smoothing (finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns_nn)
train_config.rnn_name = "lstm"

train_config.pop("mixup")
train_config.label_smooth = {
    "prob": 0.8,
    "smoothing": 0.1
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train_nn, ds_val_nn)

In [ ]:
trainer.augmenter_manager

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

## 12 lead, resnet_nature_comm_se, se, 2-linears, weightedBCE, lr=1e-4 to 2e-3, one cycle (finished)

In [ ]:
cw = np.zeros((len(train_config.class_weights),), dtype=np.float32)
for idx, c in enumerate(train_config.classes):
    cw[idx] = train_config.class_weights[c]
class_weights = torch.from_numpy(cw.astype(np.float32)).view(1, len(train_config.classes))
class_weights

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.loss = "BCEWithLogitsWithClassWeightLoss"
train_config.loss_kw = {
    "class_weight": class_weights
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.augmenter_manager

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

## 12 lead, tresnetN, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.cnn_name = "tresnetN"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.augmenter_manager

In [ ]:
trainer.train()

## 12 lead, tresnetP, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle(finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.cnn_name = "tresnetP"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

## 12 lead, tresnetF, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle(finished)

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.cnn_name = "tresnetF"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model

In [ ]:
from cfg import six_leads, four_leads, three_leads, two_leads

## 4 leads, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.leads = deepcopy(four_leads)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
ds_train.to(four_leads)
ds_val.to(four_leads)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _

## 3 leads, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.leads = deepcopy(three_leads)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
ds_train.to(three_leads)
ds_val.to(three_leads)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:

import logging
logging.shutdown()
del trainer
del model
del _

## 2 leads, resnet_nature_comm_se, lstm, se, 2-linears, AsymmetricLoss, lr=1e-4 to 2e-3, one cycle

In [ ]:
train_config = deepcopy(TrainCfg_ns)
train_config.rnn_name = "lstm"
train_config.leads = deepcopy(two_leads)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_config.n_leads = len(train_config.leads)

tranches = train_config.tranches_for_training
if tranches:
    classes = train_config.tranche_classes[tranches]
else:
    classes = train_config.classes

if train_config.n_leads == 12:
    model_config = deepcopy(ModelCfg_ns.twelve_leads)
elif train_config.n_leads == 6:
    model_config = deepcopy(ModelCfg_ns.six_leads)
elif train_config.n_leads == 4:
    model_config = deepcopy(ModelCfg_ns.four_leads)
elif train_config.n_leads == 3:
    model_config = deepcopy(ModelCfg_ns.three_leads)
elif train_config.n_leads == 2:
    model_config = deepcopy(ModelCfg_ns.two_leads)
model_config.cnn.name = train_config.cnn_name
model_config.rnn.name = train_config.rnn_name
model_config.attn.name = train_config.attn_name

In [ ]:
model = ECG_CRNN_CINC2021(
    classes=train_config.classes,
    n_leads=train_config.n_leads,
    config=model_config,
)

In [ ]:
model.module_size_

In [ ]:
if torch.cuda.device_count() > 1:
    model = DP(model)
    # model = DDP(model)
model.to(device=device)

In [ ]:
trainer = CINC2021Trainer(
    model=model,
    model_config=model_config,
    train_config=train_config,
    device=device,
    lazy=True,
)

In [ ]:
ds_train.to(two_leads)
ds_val.to(two_leads)

In [ ]:
trainer._setup_dataloaders(ds_train, ds_val)

In [ ]:
trainer.train()

In [ ]:
del trainer
import logging
logging.shutdown()
del model
del _